In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, Column, join, MaskedColumn
from astropy import units as u
from astroquery.gaia import Gaia


/Users/mbedell/python/gr8stars/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
table = Table.read('gr8_updated.csv')

### merge with table from Freckelton+2025:

table source: https://zenodo.org/records/15441644

In [3]:
paper_tbl = Table.read('gr8stars_paper1_results_revision7.csv')

In [4]:
paper_tbl.sort(['ra_j2000', 'dec_j2000'])
table.sort(['ra_j2000', 'dec_j2000'])

In [5]:
np.where(paper_tbl['bp_rp'] != table['bp_rp']) # check that the Gaia sources are perfectly aligned

(array([], dtype=int64),)

In [6]:
mismatch = np.where(paper_tbl['primary_name'] != table['primary_name'])[0]
print(len(mismatch))
mismatch[0:10]

37


array([  44,   82,  125,  180,  376,  405,  413,  652,  935, 1055])

In [7]:
mismatch_tic = np.where(paper_tbl['tic'] != table['tic'])[0]
mismatch_hd = np.where(paper_tbl['hd'] != table['hd'])[0]
print(len(mismatch_tic), len(mismatch_hd))
mismatch_tic, mismatch_hd

9 6


(array([ 123,  413,  905, 1106, 1297, 1353, 1361, 2033, 2181]),
 array([  82,  125,  376,  413, 1576, 2153]))

#### Notes on mismatches:

- All TIC mismatches are missing in one table or the other, can merge by accepting the non-masked one.
- HD mismatches are actual disagreements, fortunately none have spectra so file naming shouldn't be an issue:
    -  HD4277/HD4312 the paper table is the correct one
    -  all others are binaries/double stars and it's really not clear which component is correct, I'll default to agree with Alix/SIMBAD
- Primary name mismatches:
    - many are the binary confusion again
    - seems like for stars without HD sources but with HIP sources the paper table doesn't use HIP
    - stars with spectra:
        - GaiaEDR3892215482207937152/HIP36357 --> HIP goes with the file name
        - HD71974A/HD71974 --> HD71974 goes with the file name, and also agrees with SIMBAD's DR3 match
        - pi01UMa/pi.01UMa --> pi.01UMa goes with the file name and is closer to SIMBAD
        - HD95735/GaiaEDR3762815470562110464 --> HD is correct, but for some reason neither table has an entry in the HD column for this one, so need to fix that too. rename spectrum?
        - GaiaEDR31278392381386793856/HIP74674 --> HIP agrees with file name
        - GaiaEDR31325706698899339136/HIP80279 --> instrument column is wrong, should be HERMES not SOPHIE! HIP name agrees with file
    - other cases to note:
        - HD90444A/GaiaEDR33890250298609911168 is missing the HD match in both tables, similar to the case for HD95735
        - 16CygA/16CygC --> should definitely be 16 Cyg A, don't know how that went wrong. other identifiers are correct.

#### Actions taken:
- For the 8 stars where the paper table had a TIC and the web table didn't, took the paper's TIC ID
- For primary name, mostly used the web table, with the exceptions:
    - GaiaEDR33890250298609911168 becomes HD90444A -- also its HD column gets filled in, and HIP and TIC are adopted from paper table
    - GaiaEDR3762815470562110464 becomes HD95735 (and spectrum is renamed!)  -- also its HD column gets filled in, and HIP and TIC are adopted from paper table
    - 16CygC becomes 16CygA
    - HD4312 becomes HD4277 -- also its HD becomes 4277 and its HIP becomes 3589. TIC and Gaia are good already.
    - HIP80279 instrument gets changed to HERMES

In [8]:
table['tic'][mismatch_tic] = np.where(table['tic'][mismatch_tic].mask, paper_tbl['tic'][mismatch_tic].mask, table['tic'][mismatch_tic])

gaia_id = 3890250298609911168
table['primary_name'][table['gaiaedr3_source_id'] == gaia_id] = 'HD90444A'
table['hd'][table['gaiaedr3_source_id'] == gaia_id] = '90444A'
table['hip'][table['gaiaedr3_source_id'] == gaia_id] = paper_tbl['hip'][table['gaiaedr3_source_id'] == gaia_id]
table['tic'][table['gaiaedr3_source_id'] == gaia_id] = paper_tbl['tic'][table['gaiaedr3_source_id'] == gaia_id]

gaia_id = 762815470562110464
table['primary_name'][table['gaiaedr3_source_id'] == gaia_id] = 'HD95735'
table['hd'][table['gaiaedr3_source_id'] == gaia_id] = '95735'
table['hip'][table['gaiaedr3_source_id'] == gaia_id] = paper_tbl['hip'][table['gaiaedr3_source_id'] == gaia_id]
table['tic'][table['gaiaedr3_source_id'] == gaia_id] = paper_tbl['tic'][table['gaiaedr3_source_id'] == gaia_id]

table['primary_name'][table['primary_name'] == '16CygC'] = '16CygA'

table['primary_name'][table['primary_name'] == 'HD4312'] = 'HD4277'
table['hd'][table['primary_name'] == 'HD4277'] = '4277'
table['hip'][table['primary_name'] == 'HD4277'] = paper_tbl['hip'][table['primary_name'] == 'HD4277']

paper_tbl['Instrument'][paper_tbl['primary_name'] == 'GaiaEDR31325706698899339136'] = 'HERMES'

### construct new table with Gaia parameters:

In [9]:
gr8 = table['ra_j2000','dec_j2000','primary_name']
gr8.rename_column('primary_name','primary')
gr8['simbad'] = paper_tbl['s_name']
gr8['gaiadr2_source_id'] = table['gaiadr2_source_id']
gr8['gaiadr3_source_id'] = table['gaiaedr3_source_id']
gr8.add_columns([table['hip'],table['hd'],table['tic'],table['2mass'],table['wisea']])

In [10]:
# gaia DR3:
gr8['primary','gaiadr3_source_id'].write('gaiadr3.vot', format='votable', overwrite=True)  # write out the DR3 source_ids
#Gaia.ROW_LIMIT = 5000
query = "SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr3.gaia_source AS gaia WHERE tbl.gaiadr3_source_id=gaia.source_id"
j = Gaia.launch_job(query=query, upload_resource='gaiadr3.vot', 
                    upload_table_name="tbl", verbose=True)
gaiadr3 = j.get_results()

Launched query: 'SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr3.gaia_source AS gaia WHERE tbl.gaiadr3_source_id=gaia.source_id'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = multipart/form-data; boundary====1753711806507===
200 200
[('Date', 'Mon, 28 Jul 2025 14:10:06 GMT'), ('Server', 'Apache/2.4.6 (SLES Expanded Support platform 7) OpenSSL/1.0.2k-fips mod_jk/1.2.43'), ('Set-Cookie', 'JSESSIONID=73055A41FC5E96A793DC7FE21E47B23A; Path=/tap-server; Secure; HttpOnly'), ('Content-Encoding', 'gzip'), ('Content-Disposition', 'attachment;filename="1753711806867O-result.vot.gz"'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Transfer-Encoding', 'chunked'), ('Content-Type', 'application/x-votable+xml')]
Retrieving sync. results...
Query finished.


In [11]:
gr8.add_columns([gaiadr3['ra'],gaiadr3['ra_error'],gaiadr3['dec'],gaiadr3['dec_error'],gaiadr3['parallax'],gaiadr3['parallax_error']])
gr8.add_columns([gaiadr3['pmra'],gaiadr3['pmra_error'],gaiadr3['pmdec'],gaiadr3['pmdec_error'],gaiadr3['ref_epoch'],gaiadr3['ruwe']])
gr8['gmag'] = gaiadr3['phot_g_mean_mag']
gr8.add_columns([gaiadr3['bp_rp'],gaiadr3['radial_velocity'],gaiadr3['radial_velocity_error'],gaiadr3['vbroad'],gaiadr3['vbroad_error']])

In [12]:
gr8.add_columns([gaiadr3['teff_gspphot'],gaiadr3['teff_gspphot_lower'],gaiadr3['teff_gspphot_upper']])
gr8.add_columns([gaiadr3['logg_gspphot'],gaiadr3['logg_gspphot_lower'],gaiadr3['logg_gspphot_upper']])
gr8.add_columns([gaiadr3['mh_gspphot'],gaiadr3['mh_gspphot_lower'],gaiadr3['mh_gspphot_upper']])
gr8.add_columns([gaiadr3['distance_gspphot'],gaiadr3['distance_gspphot_lower'],gaiadr3['distance_gspphot_upper']])

In [13]:
# gaia DR3 parameters:
query = "SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr3.astrophysical_parameters AS gaia WHERE tbl.gaiadr3_source_id=gaia.source_id"
j = Gaia.launch_job(query=query, upload_resource='gaiadr3.vot', 
                    upload_table_name="tbl", verbose=True)
dr3par = j.get_results()

Launched query: 'SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr3.astrophysical_parameters AS gaia WHERE tbl.gaiadr3_source_id=gaia.source_id'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = multipart/form-data; boundary====1753711809656===
200 200
[('Date', 'Mon, 28 Jul 2025 14:10:10 GMT'), ('Server', 'Apache/2.4.6 (SLES Expanded Support platform 7) OpenSSL/1.0.2k-fips mod_jk/1.2.43'), ('Set-Cookie', 'JSESSIONID=3834477257588FC0B68E0118D57F7729; Path=/tap-server; Secure; HttpOnly'), ('Content-Encoding', 'gzip'), ('Content-Disposition', 'attachment;filename="1753711810019O-result.vot.gz"'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Transfer-Encoding', 'chunked'), ('Content-Type', 'application/x-votable+xml')]
Retrieving sync. results...


Query finished.


In [14]:
# gaia DR3 parameters:
query = "SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr3.astrophysical_parameters AS gaia WHERE tbl.gaiadr3_source_id=gaia.source_id"
j = Gaia.launch_job(query=query, upload_resource='gaiadr3.vot', 
                    upload_table_name="tbl", verbose=True)
dr3par = j.get_results()

Launched query: 'SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr3.astrophysical_parameters AS gaia WHERE tbl.gaiadr3_source_id=gaia.source_id'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = multipart/form-data; boundary====1753711812894===
200 200
[('Date', 'Mon, 28 Jul 2025 14:10:13 GMT'), ('Server', 'Apache/2.4.6 (SLES Expanded Support platform 7) OpenSSL/1.0.2k-fips mod_jk/1.2.43'), ('Set-Cookie', 'JSESSIONID=B0458F679952ECBBD62AE9FBE09382F6; Path=/tap-server; Secure; HttpOnly'), ('Content-Encoding', 'gzip'), ('Content-Disposition', 'attachment;filename="1753711813253O-result.vot.gz"'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Transfer-Encoding', 'chunked'), ('Content-Type', 'application/x-votable+xml')]
Retrieving sync. results...
Query finished.


In [15]:
assert np.all(gr8['teff_gspphot'] == dr3par['teff_gspphot']) # just checking alignment
gr8.add_columns([dr3par['teff_gspspec'], dr3par['teff_gspspec_lower'], dr3par['teff_gspspec_upper']])
gr8.add_columns([dr3par['logg_gspspec'], dr3par['logg_gspspec_lower'], dr3par['logg_gspspec_upper']])
gr8.add_columns([dr3par['mh_gspspec'], dr3par['mh_gspspec_lower'], dr3par['mh_gspspec_upper']])

### add paper parameters:

In [17]:
paper_tbl.columns

<TableColumns names=('ra_j2000','dec_j2000','primary_name','gaiadr3_source_id','hip','hd','tic','parallax','pmra','pmdec','gmag','bp_rp','radial_velocity','spectrum_harps_1d','spectrum_harps_2d','spectrum_sophie_1d','spectrum_sophie_2d','Teff(K)','Teff_err(K)','logg','logg_err','Fe/H','Fe/H_err','vmic','vmic_err','vmac','vmac_err','vsini','vsini_err','Instrument','gdr3_teff','gdr3_teff_err','gdr3_logg','gdr3_logg_err','gdr3_feh','gdr3_feh_err','w1mpro','w1sigmpro','w2mpro','w2sigmpro','w3mpro','w3sigmpro','j_m','j_msigcom','h_m','h_msigcom','k_m','k_msigcom','Bmag','e_Bmag','Vmag','e_Vmag','U','eU','V','eV','W','eW','likely_membership','membership_probability','OTYPE','s_name','Planet_host','SB2','A_550nm','chisq_SED','npts_SED','radius_SED','radius_SED_ubound','radius_SED_lbound','Teff_SED','Teff_SED_ubound','Teff_SED_lbound','filtered_otypes_flag','good_flag')>

In [18]:
# PAWS params:
gr8['teff_paws'] = paper_tbl['Teff(K)']
gr8['teff_error_paws'] = paper_tbl['Teff_err(K)']
gr8['logg_paws'] = paper_tbl['logg']
gr8['logg_error_paws'] = paper_tbl['logg_err']
gr8['feh_paws'] = paper_tbl['Fe/H']
gr8['feh_error_paws'] = paper_tbl['Fe/H_err']
gr8['vmic_paws'] = paper_tbl['vmic']
gr8['vmic_error_paws'] = paper_tbl['vmic_err']
gr8['vmac_paws'] = paper_tbl['vmac']
gr8['vmac_error_paws'] = paper_tbl['vmac_err']
gr8['vsini_paws'] = paper_tbl['vsini']
gr8['vsini_error_paws'] = paper_tbl['vsini_err']
gr8['instrument_paws'] = paper_tbl['Instrument']

In [19]:
# SED params:
gr8['teff_sed'] = paper_tbl['Teff_SED']
gr8['teff_upper_sed'] = paper_tbl['Teff_SED_ubound']
gr8['teff_lower_sed'] = paper_tbl['Teff_SED_lbound']
gr8['radius_sed'] = paper_tbl['radius_SED']
gr8['radius_upper_sed'] = paper_tbl['radius_SED_ubound']
gr8['radius_lower_sed'] = paper_tbl['radius_SED_lbound']
gr8['a550_sed'] = paper_tbl['A_550nm']
gr8['npts_sed'] = paper_tbl['npts_SED']
gr8['chisq_sed'] = paper_tbl['chisq_SED']

In [20]:
# other misc flags:
gr8['membership_group'] = paper_tbl['likely_membership']
gr8['membership_probability'] = paper_tbl['membership_probability']
gr8['otype'] = paper_tbl['OTYPE']
gr8['sb2'] = paper_tbl['SB2']
gr8['planet_host'] = paper_tbl['Planet_host']

In [28]:
gr8.write('gr8_without_spectra.csv')